In [1]:
import sys
import os
import pandas as pd 
import numpy as np
import os 
import sys 
from code.data_processing import get_all_datasets
import matplotlib.pyplot as plt
job = 'fastbag_ham'
experiment_name = 'fastbag'
from ResultHandling import ResultHandling
print(get_all_datasets())

['cleveland', 'ionosphere', 'ecoli', 'iris', 'mammo_graphic', 'wisconsin_breast_cancer', 'australia', 'postop', 'yeast', 'spec']


# Script to save all box plots

In [2]:
RH = ResultHandling(job, experiment_name, height=10, width = 10, box_label_size=30, target_dir='results_finished')

if 1:
    for ds in get_all_datasets():
        # Per model train
        plot_labels, plots = [], []
        for model_nam in RH.model_names:
            plots.append(RH.get_cond_data(mgen=False, train=False, model_name=model_nam, dataset_name=ds,col='full_acc'))
            plot_labels.append(model_nam)
        RH.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_train')
        
        # Per model test
        plot_labels, plots = [], []
        for model_nam in RH.model_names:
            plots.append(RH.get_cond_data(mgen=False, train=True, model_name=model_nam, dataset_name=ds,col='full_acc'))
            plot_labels.append(model_nam)
        RH.box_plot(boxes=plots, labels=plot_labels, save=True, draw=False, figname=f'{job}_{ds}_test')
        



# Script for generating table and wilson cox test

In [3]:
f = RH.get_model_vs_dataset_dataframe(training=True)
print(f.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  fastbaggp &  ham\_fastbaggp \\
\midrule
cleveland               &   0.845411 &       0.853462 \\
ionosphere              &   0.944762 &       0.935374 \\
ecoli                   &   0.342270 &       0.319007 \\
iris                    &   0.972381 &       0.973333 \\
mammo\_graphic           &        NaN &       0.830350 \\
wisconsin\_breast\_cancer &   0.963808 &       0.962343 \\
australia               &   0.858316 &       0.858730 \\
postop                  &   0.836612 &       0.833333 \\
yeast                   &   0.294669 &       0.297752 \\
spec                    &   0.755556 &       0.752867 \\
\bottomrule
\end{tabular}



In [4]:
f = RH.get_model_vs_dataset_dataframe(training=False)
print(f.to_latex())

\begin{tabular}{lrr}
\toprule
{} &  fastbaggp &  ham\_fastbaggp \\
\midrule
cleveland               &   0.772963 &       0.768519 \\
ionosphere              &   0.914151 &       0.900314 \\
ecoli                   &   0.334983 &       0.320462 \\
iris                    &   0.940741 &       0.942222 \\
mammo\_graphic           &        NaN &       0.814993 \\
wisconsin\_breast\_cancer &   0.948780 &       0.951057 \\
australia               &   0.838969 &       0.845089 \\
postop                  &   0.619753 &       0.601235 \\
yeast                   &   0.285426 &       0.291106 \\
spec                    &   0.659259 &       0.681481 \\
\bottomrule
\end{tabular}



In [5]:
RH.wilcoxon_test()

['ham_fastbaggp', 'fastbaggp', RanksumsResult(statistic=0.13956631834386188, pvalue=0.5554986729168594)]


In [6]:
q = np.array([1,2,3])
print(np.mean(q))
print(type(float(np.mean(q))))

2.0
<class 'float'>


# Fast GP

In [7]:
RH.model_names

['fastbaggp', 'ham_fastbaggp']